In [1]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp


In [2]:
project_releases = pd.read_csv('complete_links_allVersions_limit20.csv')

In [3]:
project_releases.head()

,project_name,project_link,version_name,commit,major_version_x,major_version_int,major_version_y,count,rank
0,web3j-web3j,https://github.com/web3j/web3j,v4.6.1,"Commit(sha=""8e94d2bcf305358d030bab9f89d4692bb3...",4,4.0,4,30,1.0
1,web3j-web3j,https://github.com/web3j/web3j,v4.6.0-android,"Commit(sha=""9ff030a628caf2c3a77e8952bbf7feb890...",4,4.0,4,30,1.0
2,web3j-web3j,https://github.com/web3j/web3j,v4.6.0,"Commit(sha=""5877c308e11c0de8e24e50c882effa8435...",4,4.0,4,30,1.0
3,web3j-web3j,https://github.com/web3j/web3j,v4.5.9,"Commit(sha=""6c23f373ce8c30a38331c4eebf46517be5...",4,4.0,4,30,1.0
4,web3j-web3j,https://github.com/web3j/web3j,v4.5.8,"Commit(sha=""62df9d1c91281664c8534b994a4bbf0941...",4,4.0,4,30,1.0


In [4]:
#project_releases = project_releases[project_releases['project_name'] == 'JodaOrg-joda-time']

In [ ]:
finished_project_arr = []
finished_project = open('finished_projects.txt','r')
for line in finished_project:
    finished_project_arr.append(line.split('\n')[0])
finished_project.close()
for project_name in project_releases['project_name'].unique():
    max_value_rsf = 0
    if project_name not in finished_project_arr:
        current_project = pd.DataFrame(project_releases[project_releases['project_name'] == project_name])
        current_project['rank'] = current_project['version_name'].rank()
        current_project = current_project.sort_values(by=['rank'], ascending=False)

        
        ### Download dataset
        try:
            os.mkdir('raw_sourcecode/' + project_name)
        except:
            pass
        for row in current_project.iterrows():
            command = 'cd C:/Users/user/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name +' & mkdir ' + project_name + '_' +row[1]['version_name']

            #print(command)
            os.system(command)

            command = 'git clone ' + row[1]['project_link'] +  ' C:/Users/user/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name + '_' + row[1]['version_name']

            #print(command)
            os.system(command)
            
            commit = row[1]['commit'].replace('Commit(sha="','')
            commit = commit.replace('")','')
            
            command = 'cd C:/Users/user/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name +'_' + row[1]['version_name'] + ' & git checkout ' + commit
            #print(command)
            os.system(command)


        ### Run Depends
        try:
            os.mkdir('raw_depends/' + project_name)
        except:
            pass
        for row in current_project.iterrows():
            #command = 'cd C:/Users/user/Desktop/SoftwareRemodularization/raw_depends/' + project_name +' & mkdir ' + project_name + '_' +row[1]['version_name']
            #os.system(command)
            #print(command)
            command = 'cd C:/Users/user/Desktop/SoftwareRemodularization/depends-0.9.2 & ' + 'java -jar depends.jar java C:/Users/user/Desktop/SoftwareRemodularization/raw_sourcecode/'  + project_name + '/' + project_name + '_' + row[1]['version_name']+  ' ../raw_depends/' + project_name + "/" + project_name + '_' + row[1]['version_name'] 
            #print(command)
            os.system(command)

        
        try:
            os.mkdir('groundtruth/' + project_name)
        except:
            pass

        try:
            os.mkdir('MoJo_1.2.1/' + project_name)
        except:
            pass


        ### Counter to only ensure the top 10 projects are selected
        counter = 10

        for row in current_project.iterrows():
            
            counter -= 1
            if counter < 0:
                break
            project_rank = row[1]['rank']
            version_name = row[1]['version_name']
            #print(project_rank)
            rootdir = 'C:/Users/user/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name + '_' + row[1]['version_name']

            #print(rootdir)


            ### To obtain the current initial directory
            full_dir_arr = []
            for root, dirs, files in os.walk(rootdir):
                #print(root)
                #print(dirs)
                for element in files:
                    if '.java' in element:
                        dir_string = root + '\\' + element
                        full_dir_arr.append(dir_string)

            cluster_dict = {}
            cluster_tree = {}


            for element in full_dir_arr:
                element = element.split('\\')
                child = element[-1]
                parent = element[-2]
                cluster_tree[child] = parent

            current_rank = project_rank
            while current_rank >= (project_rank - 9):
                current_rank -= 1
                project_to_be_compared = current_project[current_project['rank'] == current_rank]
                project_to_be_compared_rootdir = 'C:/Users/user/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name + '_' + project_to_be_compared['version_name'].values[0]
                #print(project_to_be_compared_rootdir)


                ### To obtain the current initial directory
                project_to_be_compared_full_dir_arr = []
                for root, dirs, files in os.walk(project_to_be_compared_rootdir):
                    #print(root)
                    #print(dirs)
                    for element in files:
                        if '.java' in element:
                            dir_string = root + '\\' + element
                            project_to_be_compared_full_dir_arr.append(dir_string)

                project_to_be_compared_cluster_dict = {}
                project_to_be_compared_cluster_tree = {}


                for element in project_to_be_compared_full_dir_arr:
                    element = element.split('\\')
                    child = element[-1]
                    parent = element[-2]
                    project_to_be_compared_cluster_tree[child] = parent

                cluster_tree =  {x:cluster_tree[x] for x in cluster_tree if x in project_to_be_compared_cluster_tree} 

            

            depends_dir = 'C:/Users/user/Desktop/SoftwareRemodularization/raw_depends/' + project_name + '/' + project_name + '_' + row[1]['version_name'] + '.json'
            print(depends_dir)

            with open(depends_dir) as f:
                depends_results = json.load(f)
                index  = 0
                var_array = []
                for value in depends_results['variables']:
                    var_array.append([index, value.split('\\')[-1]])
                    #print(index, value)
                    index += 1

                var_df = pd.DataFrame(var_array)
                var_df.columns = ['index_val', 'name']

                feature_list = {}
                feature_index = 2
                for element in depends_results['cells']:
                    #print(element)
                    try:
                        for a in element['values']:
                            if a not in feature_list:
                                feature_list[a] = feature_index
                                feature_index += 1
                            #print(a['Call'])


                    except:
                        pass

                feature_arr = []
                
                depends_dict = {}
                #print(len(depends_results['variables']))
                for i in range(len(depends_results['variables'])):
                    key = depends_results['variables'][i].split('\\')[-1]
                    depends_dict[key] = i
                
                arr_a_rsf = []
                filename = 'C:/Users/user/Desktop/SoftwareRemodularization/groundtruth/' + project_name + '/' + project_name + '_' + row[1]['version_name'] + '.txt'
                with open(filename, 'w') as f:
                    for key, value in cluster_tree.items():
                        try:
                            arr_a_rsf.append(depends_dict[key])
                            f.write('contain ' + str(value).replace(" ",'') + ' ' + str(key).replace(' ','') + '\n')
                        except:
                            pass
                
                for element in depends_results['cells']:
                    #print(array)
                    array = [0] * (len(feature_list) + 2)
                    values = dict(element['values'])
                    #print(element)
                    array[0] = element['src']
                    array[1] = element['dest']
                    for feature in feature_list:
                        try:
                            value = values[feature]
                            array[feature_list[feature]] = value
                        except:
                            pass
                    #print(array)
                    feature_arr.append(array)

                feature_df = pd.DataFrame(feature_arr)
                col_names = ['src', 'dest']
                for element in feature_list:
                    col_names.append(element)
                feature_df.columns = col_names

                try:
                    os.mkdir('raw_mdg/' + project_name)
                except:
                    pass

                try:
                    os.mkdir('raw_mdg_results/' + project_name)
                except:
                    pass
                test = feature_df[['src', 'dest']]

                tmp_mdg_filename = 'raw_mdg/' + project_name + '/' + project_name + '_' + version_name + '.mdg'
                #print(tmp_mdg_filename)
                tmp_mdg = open(tmp_mdg_filename, 'w+')
                for index, row in test.iterrows():
                    #print(row['src'],row['dest'])
                    tmp_mdg.write(str(row['src']) + ' ' + str(row['dest']) + '\n')

                tmp_mdg.close()

                bunch_result_filename = 'raw_mdg_results/' + project_name + '/' + project_name + '_' + version_name

                hillclimbing_filename = bunch_result_filename + '-hillclimbing'
                ga_filename = bunch_result_filename + '-ga'
                exhaustive_filename = bunch_result_filename + '-exhaustive'

                command = "java -jar alvin_bunch_automation.jar " + tmp_mdg_filename + " " + hillclimbing_filename +  " hillclimbing"
                print(command)
                os.system(command)

                command = "java -jar alvin_bunch_automation.jar " + tmp_mdg_filename + " " + ga_filename +  " ga"
                print(command)
                os.system(command)

                command = "java -jar alvin_bunch_automation.jar " + tmp_mdg_filename + " " + exhaustive_filename +  " exhaustive"
                print(command)
                os.system(command)


           
                ### This part is for hillclimbing. ###
                number = 0
                with open(hillclimbing_filename + '.bunch') as f:
                    for line in f:
                        number += 1

                filename_a = 'C:/Users/user/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + 'hillclimbing' + '_a.rsf'
                filename_b = 'C:/Users/user/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + 'hillclimbing' + '_b.rsf'

                #filename2 = 'MoJo_1.2.1/' + project_name + '/' +str(file_version) + '_' + bunch_type + '_a' +".rsf"
                bunch_counter = 0
                max_element = 0
                arr_unique = []
                arr_b_rsf = []
                with open(filename_b, 'w') as f:
                    len_b_rsf = 0
                    bunch_file = open(hillclimbing_filename + '.bunch')
                    table = []
                    for line in bunch_file:
                        bunch_counter += 1
                        line = line.replace('\n', '')
                        line = line.split(' = ')
                        line[0] = line[0].replace('SS(', '')
                        line[0] = line[0].replace(')', '')
                        line[1] = line[1].split(sep = ', ')
                        table.append(line)

                    for i in range(len(table)):
                        for j in range(len(table[i][1])):
                            #str(cluster_result[i])
                            string = "contain " + str(table[i][0]) + " " + str(table[i][1][j]) + "\n"
                            arr_unique.append(int(table[i][1][j]))
                            arr_b_rsf.append(int(table[i][1][j]))
                            if int(table[i][1][j]) > max_element:
                                max_element = int(table[i][1][j])
                            f.write(string)
                            len_b_rsf += 1
                    bunch_file.close()

                f.close()

                with open(filename_a, 'w') as f:
                    for key, value in cluster_tree.items():
                        try:
                            f.write('contain ' + str(value).replace(" ",'') + ' ' + str(depends_dict[key]).replace(' ','') + '\n')
                        except:
                            pass
                len_a_rsf = len(cluster_tree)

                to_be_added = list(set(arr_b_rsf) - set(arr_a_rsf))
                with open(filename_a,'a+') as f:
                    for item in to_be_added:
                        string = 'contain ' + str(item) + " " + str(item) + '\n'

                        f.write(string)
                f.close()

                to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
                with open(filename_b,'a+') as f:

                    for item in to_be_added:
                        string = 'contain ' + str(item) + " " + str(item) + '\n'
                        f.write(string)
                f.close()

                command = 'cd C:/Users/user/Desktop/SoftwareRemodularization/MoJo_1.2.1 & ' + 'java MoJo ' + filename_a + ' ' + filename_b + ' >> ' + project_name + '/' + project_name + '_results.txt'
                #print(command)
                os.system(command)

                f.close()

                ### This part is for ga. ###
                number = 0
                with open(ga_filename + '.bunch') as f:
                    for line in f:
                        number += 1

                filename_a = 'C:/Users/user/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + 'ga' + '_a.rsf'
                filename_b = 'C:/Users/user/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + 'ga' + '_b.rsf'

                #filename2 = 'MoJo_1.2.1/' + project_name + '/' +str(file_version) + '_' + bunch_type + '_a' +".rsf"
                bunch_counter = 0
                max_element = 0
                arr_unique = []
                arr_b_rsf = []
                with open(filename_b, 'w') as f:
                    len_b_rsf = 0
                    bunch_file = open(ga_filename + '.bunch')
                    table = []
                    for line in bunch_file:
                        bunch_counter += 1
                        line = line.replace('\n', '')
                        line = line.split(' = ')
                        line[0] = line[0].replace('SS(', '')
                        line[0] = line[0].replace(')', '')
                        line[1] = line[1].split(sep = ', ')
                        table.append(line)

                    for i in range(len(table)):
                        for j in range(len(table[i][1])):
                            #str(cluster_result[i])
                            string = "contain " + str(table[i][0]) + " " + str(table[i][1][j]) + "\n"
                            arr_unique.append(int(table[i][1][j]))
                            arr_b_rsf.append(int(table[i][1][j]))
                            if int(table[i][1][j]) > max_element:
                                max_element = int(table[i][1][j])
                            f.write(string)
                            len_b_rsf += 1
                    bunch_file.close()

                f.close()

                with open(filename_a, 'w') as f:
                    for key, value in cluster_tree.items():
                        try:
                            f.write('contain ' + str(value).replace(" ",'') + ' ' + str(depends_dict[key]).replace(' ','') + '\n')
                        except:
                            pass

                len_a_rsf = len(cluster_tree)

                to_be_added = list(set(arr_b_rsf) - set(arr_a_rsf))
                with open(filename_a,'a+') as f:
                    for item in to_be_added:
                        string = 'contain ' + str(item) + " " + str(item) + '\n'

                        f.write(string)
                f.close()

                to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
                with open(filename_b,'a+') as f:

                    for item in to_be_added:
                        string = 'contain ' + str(item) + " " + str(item) + '\n'

                        f.write(string)
                f.close()

                command = 'cd C:/Users/user/Desktop/SoftwareRemodularization/MoJo_1.2.1 & ' + 'java MoJo ' + filename_a + ' ' + filename_b + ' >> ' + project_name + '/' + project_name + '_results.txt'
                #print(command)
                os.system(command)

                f.close()

                ### This part is for exhaustive. ###
                number = 0
                with open(exhaustive_filename + '.bunch') as f:
                    for line in f:
                        number += 1

                filename_a = 'C:/Users/user/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + 'exhaustive' + '_a.rsf'
                filename_b = 'C:/Users/user/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + 'exhaustive' + '_b.rsf'

                #filename2 = 'MoJo_1.2.1/' + project_name + '/' +str(file_version) + '_' + bunch_type + '_a' +".rsf"
                bunch_counter = 0
                max_element = 0
                arr_unique = []
                arr_b_rsf = []
                with open(filename_b, 'w') as f:
                    len_b_rsf = 0
                    bunch_file = open(exhaustive_filename + '.bunch')
                    table = []
                    for line in bunch_file:
                        bunch_counter += 1
                        line = line.replace('\n', '')
                        line = line.split(' = ')
                        line[0] = line[0].replace('SS(', '')
                        line[0] = line[0].replace(')', '')
                        line[1] = line[1].split(sep = ', ')
                        table.append(line)

                    for i in range(len(table)):
                        for j in range(len(table[i][1])):
                            #str(cluster_result[i])
                            string = "contain " + str(table[i][0]) + " " + str(table[i][1][j]) + "\n"
                            arr_unique.append(int(table[i][1][j]))
                            arr_b_rsf.append(int(table[i][1][j]))
                            if int(table[i][1][j]) > max_element:
                                max_element = int(table[i][1][j])
                            f.write(string)
                            len_b_rsf += 1
                    bunch_file.close()

                f.close()

                with open(filename_a, 'w') as f:
                    for key, value in cluster_tree.items():
                        try:
                            f.write('contain ' + str(value).replace(" ",'') + ' ' + str(depends_dict[key]).replace(' ','') + '\n')
                        except:
                            pass

                len_a_rsf = len(cluster_tree)

                to_be_added = list(set(arr_b_rsf) - set(arr_a_rsf))
                with open(filename_a,'a+') as f:
                    for item in to_be_added:
                        string = 'contain ' + str(item) + " " + str(item) + '\n'

                        f.write(string)
                f.close()

                to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
                with open(filename_b,'a+') as f:

                    for item in to_be_added:
                        string = 'contain ' + str(item) + " " + str(item) + '\n'

                        f.write(string)
                f.close()

                command = 'cd C:/Users/user/Desktop/SoftwareRemodularization/MoJo_1.2.1 & ' + 'java MoJo ' + filename_a + ' ' + filename_b + ' >> ' + project_name + '/' + project_name + '_results.txt'
                #print(command)
                os.system(command)
                if max_value_rsf < len_a_rsf:
                    max_value_rsf = len_a_rsf
                if max_value_rsf < len_b_rsf:
                    max_value_rsf = len_b_rsf


                f.close()
                
                
                
                
        finished_project = open('finished_projects.txt','a+')
        finished_project.write(project_name + '\n')
        finished_project.close()

        max_len_txt = open('C:/Users/user/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_max_len.txt', "w")
        max_len_txt.write(str(max_value_rsf) + '\n')
        max_len_txt.close()
        
        shutil.copy('MoJo_1.2.1/' + project_name + '/' + project_name + '_results.txt', 'Results')
        shutil.copy('MoJo_1.2.1/' + project_name + '/' + project_name +  '_max_len.txt', 'Results')
            
            
            

C:/Users/user/Desktop/SoftwareRemodularization/raw_depends/uber-NullAway/uber-NullAway_v0.7.9.json
java -jar alvin_bunch_automation.jar raw_mdg/uber-NullAway/uber-NullAway_v0.7.9.mdg raw_mdg_results/uber-NullAway/uber-NullAway_v0.7.9-hillclimbing hillclimbing
java -jar alvin_bunch_automation.jar raw_mdg/uber-NullAway/uber-NullAway_v0.7.9.mdg raw_mdg_results/uber-NullAway/uber-NullAway_v0.7.9-ga ga
java -jar alvin_bunch_automation.jar raw_mdg/uber-NullAway/uber-NullAway_v0.7.9.mdg raw_mdg_results/uber-NullAway/uber-NullAway_v0.7.9-exhaustive exhaustive
C:/Users/user/Desktop/SoftwareRemodularization/raw_depends/uber-NullAway/uber-NullAway_v0.7.8.json
java -jar alvin_bunch_automation.jar raw_mdg/uber-NullAway/uber-NullAway_v0.7.8.mdg raw_mdg_results/uber-NullAway/uber-NullAway_v0.7.8-hillclimbing hillclimbing
java -jar alvin_bunch_automation.jar raw_mdg/uber-NullAway/uber-NullAway_v0.7.8.mdg raw_mdg_results/uber-NullAway/uber-NullAway_v0.7.8-ga ga
java -jar alvin_bunch_automation.jar raw_

C:/Users/user/Desktop/SoftwareRemodularization/raw_depends/testcontainers-testcontainers-java/testcontainers-testcontainers-java_testcontainers-1.1.8.json
java -jar alvin_bunch_automation.jar raw_mdg/testcontainers-testcontainers-java/testcontainers-testcontainers-java_testcontainers-1.1.8.mdg raw_mdg_results/testcontainers-testcontainers-java/testcontainers-testcontainers-java_testcontainers-1.1.8-hillclimbing hillclimbing
java -jar alvin_bunch_automation.jar raw_mdg/testcontainers-testcontainers-java/testcontainers-testcontainers-java_testcontainers-1.1.8.mdg raw_mdg_results/testcontainers-testcontainers-java/testcontainers-testcontainers-java_testcontainers-1.1.8-ga ga
java -jar alvin_bunch_automation.jar raw_mdg/testcontainers-testcontainers-java/testcontainers-testcontainers-java_testcontainers-1.1.8.mdg raw_mdg_results/testcontainers-testcontainers-java/testcontainers-testcontainers-java_testcontainers-1.1.8-exhaustive exhaustive
C:/Users/user/Desktop/SoftwareRemodularization/raw

In [6]:
shutil.copy('MoJo_1.2.1/apache-log4j/apache-log4j_results.txt', 'Results')
shutil.copy('MoJo_1.2.1/apache-log4j/apache-log4j_max_len.txt', 'Results')

'Results\\apache-log4j_results.txt'

In [9]:
project_name = 'apache-log4j'
command = 'git clone ' + 'https://github.com/apache/log4j' +  ' C:/Users/user/Desktop/SoftwareRemodularization/test/' + project_name + '/' + project_name + '_' + '8.1.1'
os.system(command)
command = 'cd C:/Users/user/Desktop/SoftwareRemodularization/test/' + project_name + '/' + project_name +'_' + '8.1.1' + ' & git checkout ' + 'fcbe46c'

os.system(command)

1

In [7]:
project_releases.head()

,project_name,project_link,revision,version_name
1071,apache-log4j,https://github.com/apache/log4j,31d7ec7,8.2.0
1072,apache-log4j,https://github.com/apache/log4j,fcbe46c,8.1.1
1073,apache-log4j,https://github.com/apache/log4j,dbe5ed0,8.1.0
1074,apache-log4j,https://github.com/apache/log4j,2ae4746,8.0.0
1075,apache-log4j,https://github.com/apache/log4j,d4c30fc,7.7.2
